In [1]:
import pandas as pd

In [2]:
# Function to load and clean the data
def clean_survey_data(file_path):
    # Load the data from the Excel file
    df = pd.read_excel(file_path)

    # Renaming and formatting columns

    # Convert 'First Response Date' to datetime and rename it to 'Date'
    df['Date'] = pd.to_datetime(df['First Response Date'], dayfirst=True).dt.date

    # Rename '2) Are you:' to 'Type' and adjust data
    df['Type'] = df['2) Are you:'].str.extract(r'(\w+) \[').fillna('Unknown')

    # Rename '1) Clinic:' to 'Unit' and adjust data
    df['Unit'] = df['1) Clinic:'].str.extract(r'(\w+) \[')

    # Ensure 'Average Score' is a float with two decimal places
    df['Average Score'] = df['Average Score'].astype(float).round(2)

    # Function to extract the initial integer from a string
    extract_initial_integer = lambda s: int(s.split()[0]) if pd.notna(s) else None

    # Renaming and extracting initial integer for specified columns
    df['Listen'] = df['3) How much effort did we make to listen to what’s important to you?'].apply(extract_initial_integer)
    df['Involve'] = df['4) Did we involve you in decisions about your (or your loved ones) care and treatment?'].apply(extract_initial_integer)
    df['Accessibility'] = df['5) How accessible were we when you needed us?'].apply(extract_initial_integer)
    df['Trust'] = df['6) Did you have confidence and trust in the critical care team treating you (or your loved one)?'].apply(extract_initial_integer)
    df['Overall'] = df['9) Finally, thinking about your stay on critical care, overall, what was the experience of the care you received?'].apply(extract_initial_integer)

    # Keeping qualitative questions columns (question 7, 8, & 10)
    qualitative_columns = ['7) What (or who) made a difference to you today and why?',
                           '8) What could we have done to make your day better?',
                           '10) Do you have any other comments or feedback to help us improve?']

    # Dropping all other columns except the specified ones
    columns_to_keep = ['Date', 'Type', 'Unit', 'Average Score', 'Listen', 'Involve', 'Accessibility', 'Trust', 'Overall'] + qualitative_columns
    cleaned_df = df[columns_to_keep]

    # Removing records where 'Type' is 'Unknown' or 'Average Score' is NaN or 0
    cleaned_df = cleaned_df[
        (cleaned_df['Type'] != 'Unknown') & 
        (cleaned_df['Average Score'].notna()) & 
        (cleaned_df['Average Score'] != 0) &
        (cleaned_df['Unit'].notna())
    ]
    
    return cleaned_df

In [3]:
# Define the path for the Excel file and the new CSV file
excel_file_path = 'PatientSurvey.xlsx'
csv_file_path = 'cleaned_survey.csv'

# Clean the data
cleaned_data = clean_survey_data(excel_file_path)

# Save the cleaned data to a CSV file
cleaned_data.to_csv(csv_file_path, index=False)

print("Data cleaned and saved successfully.")

/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Data cleaned and saved successfully.
